In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from linearmodels import PanelOLS
import plotly.express as px

sns.set_theme()

In [2]:
DF = pd.read_csv('Datasets/GenderStatistics/WB_GS_WIDEF.csv')
df = DF.copy()

In [3]:
df.drop(columns=['FREQ', 'FREQ_LABEL','INDICATOR','AGE','UNIT_MEASURE',
       'COMP_BREAKDOWN_1_LABEL', 'COMP_BREAKDOWN_2', 'COMP_BREAKDOWN_2_LABEL',
       'DATABASE_ID', 'DATABASE_ID_LABEL', 'UNIT_MULT', 'UNIT_MULT_LABEL',
       'OBS_STATUS', 'OBS_STATUS_LABEL', 'OBS_CONF', 'OBS_CONF_LABEL'], inplace=True)

In [4]:
df_melt = df.melt(
    id_vars=['REF_AREA', 'REF_AREA_LABEL', 'INDICATOR_LABEL', 'SEX', 'SEX_LABEL', 'UNIT_MEASURE_LABEL','AGE_LABEL','COMP_BREAKDOWN_1'],
    value_vars=[col for col in df.columns if col.isdigit()],
    var_name='YEAR',
    value_name='VALUE'
)
df_melt['YEAR'] = pd.to_datetime(df_melt['YEAR'], format='%Y', errors='coerce').dt.year
df_melt['VALUE'] = pd.to_numeric(df_melt['VALUE'], errors='coerce')
df_melt.dropna(subset=['VALUE'], inplace=True)

In [5]:
INDICATOR_NON_NEEDED = ['Inflation, consumer prices (annual %)', 'GDP (current US$)',
       'GDP growth (annual %)', 'GDP per capita (current US$)',
       'GDP per capita (constant 2010 US$)',
       'GNI, Atlas method (current US$)',
       'GNI per capita, Atlas method (current US$)',
       'GNI per capita, PPP (current international $)',
       'Children out of school, primary (Number)',
       'Educational attainment by level of education, cumulative (% population 25+)',
       'School enrollment, gender parity index',
       'School enrollment, preprimary (% gross)',
       'Primary completion rate, based on completers (%)',
       'Primary completion rate (% of relevant age group)',
       'Primary education, pupils (% female)',
       'School enrollment, primary (%)',
       'Gross intake ratio in grade 1, total (% of relevant age group)',
       'Net intake rate in grade 1 (% of official school-age population)',
       'Persistence to grade 5 (% of cohort)',
       'Primary education, teachers (% female)',
       'Adjusted net enrollment rate, primary (% of primary school age children)',
       'Children out of school, primary (Number)',
       'Children out of school (% of primary school age)',
       'Expected years of schooling',
       'Lower secondary completion rate (% of relevant age group)',
       'Secondary education, pupils (% female)',
       'Students in secondary education enrolled in vocational programmes, male (%)',
       'Secondary education, vocational pupils',
       'Secondary education, vocational pupils (% female)',
       'Vocational and technical enrolment (% of total secondary enrolment)',
       'School enrollment, secondary (%)',
       'Progression to secondary school (%)',
       'Secondary education, teachers (% female)',
       'Gross graduation ratio, tertiary (%)',
       'School enrollment, tertiary (% gross)',
       'Share of graduates by field, female (%)',
       'Tertiary education, academic staff (% female)',
       'Ratio of female to male labor force participation rate (%)',
       'Ratio of female to male youth unemployment rate (% ages 15-24)',
       'Share of youth not in education, employment or training (% of youth population)',
       'Unemployment (%)',
       'Adolescent fertility rate (births per 1,000 women ages 15-19)',
       'Mortality rate, adult (per 1,000 adults)',
       'Birth rate, crude (per 1,000 people)',
       'Death rate, crude (per 1,000 people)',
       'Life expectancy at birth (years)',
       'Life expectancy at age 60 (years)', 'Mean age at first marriage',
       'Fertility rate, total (births per woman)',
       'Survival to age 65 (% of cohort)', 'Population (number)',
       'Sex ratio at birth (male births per female births)',
       'Age dependency ratio (% of working-age population)',
       'Age population, interpolated',
       'Population, female (% of total population)',
       'Population (age group as % of total population)',
       'Students in lower secondary vocational education, female (%)',
       'Students in upper secondary vocational education, female (%)',
       'Students in post-secondary non-tertiary vocational education,  female (%)',
       'Share of graduates in Business, Administration and Law programmes, female (%)',
       'Share of graduates in other fields than Science, Technology, Engineering and Mathematics programmes, female (%)',
       'Share of all students in lower secondary education enrolled in vocational programmes (%)',
       'Share of all students in upper secondary education enrolled in vocational programmes (%)',
       'Share of all students in post-secondary non-tertiary education enrolled in vocational programmes (%)',
       'Youth illiterate population, 15-24 years, female (%)',
       'Youth illiterate population, 15-24 years (number)',
       'Rate of out-of-school youth of upper secondary school age (%)',
       'Intentional homicides (per 100,000 people)',
       'Cost of business start-up procedures (% of GNI per capita)',
       'Time required to start a business (days)',
       'Start-up procedures to register a business (number)',
       'Proportion of seats held by women in national parliaments (%)',
       'Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)',
       'Prevalence of anemia among women of reproductive age (% of women ages 15-49)',
       'Prevalence of anemia among non-pregnant women (% of women ages 15-49), with Hb<120 g/L',
       'Number of infant deaths', 'Number of under-five deaths',
       'Number of stillbirths', 'Cause of death (%)',
       'Share of population ages 15+ living with HIV, female (%)',
       'Mortality rate, under-5 (per 1,000 live births)',
       'Mortality from CVD, cancer, diabetes or CRD between exact ages 30 and 70 (%)',
       'Stillbirth rate (per 1,000 total births)',
       'Prevalence of HIV (% ages 15-24)',
       'Antiretroviral therapy coverage (% of adults living with HIV)',
       'Incidence of HIV (per 1,000 uninfected population)',
       'Antiretroviral therapy coverage for PMTCT (% of pregnant women living with HIV)',
       'Immunization, DPT (% of children ages 12-23 months)',
       'Immunization, measles (% of children ages 12-23 months)',
       'Intermittent Preventive Treatment of malaria in pregnancy (% of pregnant women)',
       'Number of maternal deaths',
       'Lifetime risk of maternal death (1 in: rate varies by country)',
       'Lifetime risk of maternal death (%)',
       'Prevalence of anemia among pregnant women (any age), with Hb<110 g/L',
       'Prevalence of syphilis (% of women attending antenatal care)',
       'Prevalence of current tobacco use (% of adults)',
       'Mortality rate attributed to household and ambient air pollution, age-standardized (per 100,000 population)',
       'Maternal mortality ratio (per 100,000 live births)',
       'People practicing open defecation, rural (% of rural population)',
       'People practicing open defecation, urban (% of urban population)',
       'Prevalence of overweight (% of adults)',
       'Mortality rate attributed to unintentional poisoning (per 100,000 population)',
       'Suicide mortality rate (per 100,000 population)',
       'Mortality caused by road traffic injury (per 100,000 population)',
       'Mortality rate attributed to unsafe water, unsafe sanitation and lack of hygiene (per 100,000 population)',
       'Employment by sector (%)',
       'Employers (% of employment) (modeled ILO estimate)',
       'Self-employed (% of employment) (modeled ILO estimate)',
       'Vulnerable employment (% of employment) (modeled ILO estimate)',
       'Wage and salaried workers (% of employment) (modeled ILO estimate)',
       'Labor force, female (% of total labor force)',
       'Labor force (number)',
       'Mortality rate, infant (per 1,000 live births)',
       'Completeness of birth registration (%)',
       'Unmet need for contraception (% of married women ages 15-49)',
       'No account because financial institutions are too far away (% age 15+)',
       'No account because financial institutions are too far away (% without an account, age 15+)',
       'No account because financial services are too expensive (% age 15+)',
       'No account because financial services are too expensive (% without an account, age 15+)',
       'No account because of a lack of necessary documentation (% age 15+)',
       'No account because of a lack of necessary documentation (% without an account, age 15+)',
       'No account because of a lack of trust in financial institutions (% age 15+)',
       'No account because of a lack of trust in financial institutions (% without an account, age 15+)',
       'No account because of religious reasons (% age 15+)',
       'No account because of religious reasons (% without an account, age 15+)',
       'No account because of insufficient funds (% age 15+)',
       'No account because of insufficient funds (% without an account, age 15+)',
       'Can use account at a bank or financial institution without help if opened (% without an account, age 15+)',
       'Used a mobile phone or the internet to send money (% age 15+)',
       'Used a mobile phone or the internet to pay bills in the past year (% age 15+)',
       'Used a mobile phone or the internet to buy something online in the past year(% age 15+)',
       'Made a digital in-store merchant payment: using a mobile phone (% age 15+)',
       'Saved to start, operate, or expand a farm or business (% age 15+)',
       'Saved for old age (% age 15+)',
       'Saved money using a mobile money account (% age 15+)',
       'Saved at a financial institution or using a mobile money account (% age 15+)',
       'Saved at a financial institution (% age 15+)',
       'Saved using a savings club or a person outside the family (% age 15+)',
       'Saved for education or school fees (% age 15+)',
       'Saved any money in the past year (% age 15+)',
       'Financial institution account (% age 15+)',
       'Borrowed for health or medical purposes (% age 15+)',
       'Borrowed from a store by buying on credit (% age 15+)',
       'Borrowed to start, operate, or expand a farm or business (% age 15+)',
       'Borrowed for education or school fees (% age 15+)',
       'Borrowed any money from a formal financial institution or using a mobile money account (% age 15+)',
       'Borrowed from a financial institution (% age 15+)',
       'Borrowed from family or friends (% age 15+)',
       'Borrowed from a savings club (% age 15+)',
       'Borrowed any money in the past year (% age 15+)',
       'Coming up with emergency funds in 30 days: possible (% age 15+)',
       'Coming up with emergency funds in 7 days: possible (% age 15+)',
       'Coming up with emergency funds in 30 days: not possible (% age 15+)',
       'Sent or received domestic remittances in the past year (% age 15+)',
       'Received domestic remittances in the past year (% age 15+)',
       'Sent domestic remittances in the past year (% age 15+)',
       'Debit card ownership (% age 15+)',
       'Paid utility bills in the past year (% age 15+)',
       'Paid school fees in the past year (% age 15+)',
       'Received government transfer or pension (% age 15+)',
       'Received government transfers in the past year (% age 15+)',
       'Received a public sector pension in the past year (% age 15+)',
       'Received payments for the sale of agricultural products, livestock, or crops (% age 15+)',
       'Worried about not having enough money for old age (% age 15+)',
       'Worried about not being able to pay for medical costs in case of a serious illness or accident (% age 15+)',
       'Worried about not having enough money for monthly expenses or bills (% age 15+)',
       'Worried about not being able to pay school fees or fees for education (% age 15+)',
       'Experience or continue to experience severe financial hardship as a result of the disruption caused by COVID-19 (% age 15+)',
       'Most worrying financial issue (% age 15+)',
       'Used a mobile phone or the internet to access an account (% with an account, age 15+)',
       'Credit card ownership (% age 15+)',
       'Has an inactive account (% age 15+)',
       'Received government payments in the past year (% age 15+)',
       'Account ownership at a financial institution or with a mobile-money-service provider (% of population ages 15+)',
       'Made or received digital payments in the past year (% age 15+)',
       'Made digital payments in the past year (% age 15+)',
       'Received digital payments in the past year (% age 15+)',
       'Survival Rate from Age 15-60', 'Expected Years of School',
       'Harmonized Test Scores', 'Learning-Adjusted Years of School',
       'Probability of Survival to Age 5',
       'Human Capital Index (HCI) (scale 0-1)',
       'Human Capital Index, Lower Bound (scale 0-1)',
       'Human Capital Index, Upper Bound (scale 0-1)',
       'Fraction of Children Under 5 Not Stunted', 'Number of directors',
       'Mobile money account (% age 15+)',
       'Learning poverty: Share of Children at the End-of-Primary age below minimum reading proficiency adjusted by Out-of-School Children (%)',
       'Pupils below minimum reading proficiency at end of primary (%). Low GAML threshold',
       'Mandatory retirement age',
       'Main cooking fuel: charcoal (% of households)',
       'Main cooking fuel: agricultural crop (% of households)',
       'Main cooking fuel: dung (% of households)',
       'Main cooking fuel: electricity (% of households)',
       'Location of cooking: inside the house (% of households)',
       'Main cooking fuel: LPG/natural gas/biogas (% of households)',
       'Location of cooking: other places (% of households)',
       'Location of cooking: outdoors (% of households)',
       'Location of cooking: separate building (% of households)',
       'Main cooking fuel: straw/shrubs/grass (% of households)',
       'Main cooking fuel: wood (% of households)',
       'Households with water on the premises (%)',
       'Households with water less than 30 minutes away round trip (%)',
       'Households with water 30 minutes or longer away round trip (%)',
       'Proportion who initiated sexual intercourse by or before age 15',
       'Proportion of women subjected to physical and/or sexual violence in the last 12 months (%)',
       'Proportion of women who have ever experienced any form of sexual violence (% of women age 15-49)',
       'Proportion of women who have never sought help to stop violence, and never told anyoneÂ\xa0 (% of ever-married women ages 15-49 who have ever experienced any physical or sexual violence)',
       'Proportion of women who have never sought help to stop violence, but told someone (% of ever-married women ages 15-49 who have ever experienced any physical or sexual violence)',
       'Proportion of women who have sought help to stop physical or sexual violence (% of women age 15-49)',
       'Comprehensive correct knowledge of HIV/AIDS (2 prevent ways and reject 3 misconceptions)',
       'Length of paid leave (calendar days)',
       'Length of parental leave (calendar days)',
       'Prevalence of underweight, weight for age (% of children under 5)',
       'Prevalence of obesity (% of population ages 18+)',
       'Prevalence of overweight, weight for height (% of children under 5)',
       'Prevalence of stunting, height for age (% of children under 5)',
       'Prevalence of wasting, weight for height (% of children under 5)',
       'Prevalence of severe wasting, weight for height (% of children under 5)',
       'Poverty headcount ratio at national poverty lines (% of population)',
       'Time-related underemployment (% of employment)',
       'Children in employment (% of children ages 7-14)',
       'Part time employment (% of employment)',
       'Labor force by level of education (%)',
       'Unemployment by level of education (%)',
       'Wanted fertility rate (births per woman)',
       'Retirement age by type of benefits',
       'Condom use at last high-risk sex (% ages 15-49)',
       'Poverty headcount ratio at $2.15 a day (2017 PPP) (% of population)',
       'Gini index (World Bank estimate)',
       'Informal employment (% of total non-agricultural employment)',
       'Share of graduates in Information and Communication Technologies programmes, female (%)',
       'Withdrew money from a financial institution account 2 or more times a month (% age 15+)',
       'Withdrew money from a financial institution account 2 or more times a month (% who had withdrawn money, age 15+)',
       'Made a digital in-store merchant payment for the first time after COVID-19 started (% age 15+)',
       'Made a digital in-store merchant payment for the first time after COVID-19 started (% who made a digital in-store payment, age 15+)',
       'Proportion of time spent on unpaid domestic and care work (% of 24 hour day)',
       'Use a mobile money account two or more times a month (% age 15+)',
       'Use a mobile money account two or more times a month (% with a mobile money account, age 15+)',
       'Can use a mobile money account without help from anyone, including a mobile money agent (% age 15+)',
       'Can use a mobile money account without help from anyone, including a mobile money agent (% with a mobile money account, age 15+)',
       'Primary school age children out-of-school (%)',
       'Made a digital online payment for an online purchase for the first time after COVID-19 started (% age 15+)',
       'Made a utility payment: using an account for the first time after COVID-19 started (% age 15+)',
       'Reason for not having a mobile money account: mobile money agents are too far away (% age 15+)',
       'Reason for not having a mobile money account: mobile money agents are too far away (% without an account, age 15+)',
       'Reason for not having a mobile money account: available mobile money products are too expensive (% age 15+)',
       'Reason for not having a mobile money account: available mobile money products are too expensive (% without an account, age 15+)',
       "Reason for not having a mobile money account: don't have the necessary documentation (% age 15+)",
       "Reason for not having a mobile money account: don't have the necessary documentation (% without an account, age 15+)",
       "Reason for not having a mobile money account: don't have enough money to use a mobile money account (% age 15+)",
       "Reason for not having a mobile money account: don't have enough money to use a mobile money account (% without an account, age 15+)",
       'Reason for not having a mobile money account: use an agent or someone else to make payments (% age 15+)',
       'Reason for not having a mobile money account: use an agent or someone else to make payments (% without an account, age 15+)',
       'Reason for not having a mobile money account: do not have their own mobile phone (% age 15+)',
       'Reason for not having a mobile money account: do not have their own mobile phone (% without an account, age 15+)',
       'Reason for not using their inactive account: bank or financial institution is too far away (% age 15+)',
       'Reason for not using their inactive account: bank or financial institution is too far away (% with an inactive account, age 15+)',
       'Reason for not using their inactive account: no need for an account (% age 15+)',
       'Reason for not using their inactive account: no need for an account (% with an inactive account, age 15+)',
       "Reason for not using their inactive account: don't have enough money to use an account (% age 15+)",
       "Reason for not using their inactive account: don't have enough money to use an account (% with an inactive account, age 15+)",
       "Reason for not using their inactive account: don't feel comfortable using the account by themselves (% age 15+)",
       "Reason for not using their inactive account: don't feel comfortable using an account by themselves (% with an inactive account, age 15+)",
       "Reason for not using their inactive account: don't trust banks or financial institutions (% age 15+)",
       "Reason for not using their inactive account: don't trust banks or financial institutions (% with an inactive account, age 15+)",
       'Literacy rate, gender parity index (youth ages 15-24)',
       'Migrants, female (% of international migrant stock)',
       'People practicing open defecation (% of population)',
       'Employment to population ratio (%)',
       'Labor force participation rate (% of population)',
       'Rural population (%)', 'Urban population (%)',
       'Contraceptive prevalence (% of women ages 15-49)',
       'Pregnant women receiving prenatal care of at least four visits (% of pregnant women)',
       'Number of business owners',
       'Number of sole proprietors',
       'Women Business and the Law Index Score (scale 1-100)',
       'Women, Business and the Law: Assets Indicator Score (scale 1-100)',
       'Women, Business and the Law: Entrepreneurship Indicator Score (scale 1-100)',
       'Women, Business and the Law: Mobility Indicator Score (scale 1-100)',
       'Women, Business and the Law: Marriage Indicator Score (scale 1-100)',
       'Women, Business and the Law: Pension Indicator Score (scale 1-100)',
       'Women, Business and the Law: Parenthood Indicator Score (scale 1-100)',
       'Women, Business and the Law: Pay Indicator Score (scale 1-100)',
       'Women, Business and the Law: Workplace Indicator Score (scale 1-100)',
       'The law provides for the valuation of nonmonetary contributions (1=yes; 0=no)',
       'Women whose first experience of spousal physical or sexual violence was before marriageÂ\xa0 (% of currently married women age 15-49 who have been married only once)',
       'Women whose first experience of spousal physical or sexual violence was during marriageÂ\xa0 (% of currently married women age 15-49 who have been married only once)',
       'Women who have not experienced spousal physical or sexual violence (% of currently married women age 15-49 who have been married only once)',
       'Proportion of women who have ever experienced intimate partner violence (%)',
       'Women who have ever experienced violence committed by their husband/partner by type of violenceÂ\xa0 (% of ever-married women ages 15-49)',
       'Women who have experienced violence committed by their husband/partner in the last 12 months by type of violence (% of ever-married women ages 15-49)',
       'Women who experienced first sexual violence before age 15, 18, or 22 (% of women ages 15-49)',
       'Knowledge of contraception methods (%)',
       'Fertility Planning Status (% of births)']

These indicators, even though they migh be interesting, are basically empty

In [6]:
INDICATOR_EMPTY = ['Demand for family planning satisfied (% of married women with demand for family planning)',
       'Female genital mutilation prevalence (%)', 'Ownership of a house (%)',
       'Ownership of land (%)',
       'Women and girls who have private places to wash and change during menstrual period (% of women and girls ages 15-49 who had a menstrual period within the last year)',
       'Women and girls who participate in activities during menstrual period (% of women and girls ages 15-49 who had a menstrual period within the last year)',
       'Women and girls who use menstrual materials (% of women and girls ages 15-49 who had a menstrual period within the last year)',
       'Women participating in decisions related to health care, purchases, visiting family, and cooking (%)',
       'Women who believe a husband is justified in beating his wife (%)',
       'Women who believe a wife is justified refusing sex with her husband (%)']

In [7]:
df2 = df_melt[~df_melt['INDICATOR_LABEL'].isin(INDICATOR_NON_NEEDED)]
df2 = df2[~df2['INDICATOR_LABEL'].isin(INDICATOR_EMPTY)]
df2 = df2.drop(columns=['UNIT_MEASURE_LABEL','SEX_LABEL','REF_AREA_LABEL'])
df2 = df2[df2['AGE_LABEL'] != 'under 15 years old']
df2 = df2[df2['AGE_LABEL'] != '15 to 24 years old']

df2 = df2[~((df2['INDICATOR_LABEL'] == 'Government expenditure on education (% of GDP)') & (df2['COMP_BREAKDOWN_1'] != 'ISIC4_P'))]
df2 = df2.drop(columns=['AGE_LABEL','COMP_BREAKDOWN_1'])

df_T = df2[df2['SEX'] == '_T']
df_F = df2[df2['SEX'] == 'F']

In [8]:
GS_T = df_T.pivot(
    index=['REF_AREA', 'YEAR'],
    columns='INDICATOR_LABEL',
    values='VALUE'
)
GS_T.columns.name = None
GS_T.reset_index(inplace=True)
GS_F = df_F.pivot(
    index=['REF_AREA', 'YEAR'],
    columns='INDICATOR_LABEL',
    values='VALUE'
)
GS_F.reset_index(inplace=True)
GS_F.columns.name = None

In [9]:
min_non_null = int(0.75 * GS_T.shape[0])
GS_T = GS_T.dropna(axis=1, thresh=min_non_null)
GS_T = GS_T.dropna()

In [10]:
min_non_null = int(0.1 * GS_F.shape[0])
GS_F = GS_F.dropna(axis=1, thresh=min_non_null)
GS_F = GS_F.dropna()

In [11]:
GS_F.isnull().sum()

REF_AREA                                                  0
YEAR                                                      0
Employment in senior and middle management, female (%)    0
Literacy rate (%)                                         0
Pregnant women receiving prenatal care (%)                0
Proportion of women in ministerial level positions (%)    0
dtype: int64

In [13]:
GS_F.shape

(28, 6)

In [16]:
GS_T.to_csv('Datasets/GenderStatistics/GS.csv')

In [15]:
GS_T

,REF_AREA,YEAR,A woman can apply for a passport in the same way as a man (1=yes; 0=no),A woman can be head of household in the same way as a man (1=yes; 0=no),A woman can choose where to live in the same way as a man (1=yes; 0=no),A woman can get a job in the same way as a man (1=yes; 0=no),A woman can obtain a judgment of divorce in the same way as a man (1=yes; 0=no),A woman can open a bank account in the same way as a man (1=yes; 0=no),A woman can register a business in the same way as a man (1=yes; 0=no),A woman can sign a contract in the same way as a man (1=yes; 0=no),...,The age at which men and women can retire with partial pension benefits is the same (1=yes; 0=no),The government administers 100% of maternity leave benefits (1=yes; 0=no),The law grants spouses equal administrative authority over assets during marriage (1=yes; 0=no),The law prohibits discrimination in access to credit based on gender (1=yes; 0=no),The mandatory retirement age for men and women is the same (1=yes; 0=no),There are periods of absence due to child care accounted for in pension benefits (1=yes; 0=no),There is legislation on sexual harassment in employment (1=yes; 0=no),There is legislation specifically addressing domestic violence (1=yes; 0=no),There is no legal provision that requires a married woman to obey her husband (1=yes; 0=no),There is paid parental leave (1=yes; 0=no)
55,AFG,1970,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
56,AFG,1971,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
57,AFG,1972,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
58,AFG,1973,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
59,AFG,1974,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12479,ZWE,2019,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
12480,ZWE,2020,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
12481,ZWE,2021,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
12482,ZWE,2022,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0


In [18]:
GS_T.iloc[:,2:].corr()

,A woman can apply for a passport in the same way as a man (1=yes; 0=no),A woman can be head of household in the same way as a man (1=yes; 0=no),A woman can choose where to live in the same way as a man (1=yes; 0=no),A woman can get a job in the same way as a man (1=yes; 0=no),A woman can obtain a judgment of divorce in the same way as a man (1=yes; 0=no),A woman can open a bank account in the same way as a man (1=yes; 0=no),A woman can register a business in the same way as a man (1=yes; 0=no),A woman can sign a contract in the same way as a man (1=yes; 0=no),A woman can travel outside her home in the same way as a man (1=yes; 0=no),A woman can travel outside the country in the same way as a man (1=yes; 0=no),...,The age at which men and women can retire with partial pension benefits is the same (1=yes; 0=no),The government administers 100% of maternity leave benefits (1=yes; 0=no),The law grants spouses equal administrative authority over assets during marriage (1=yes; 0=no),The law prohibits discrimination in access to credit based on gender (1=yes; 0=no),The mandatory retirement age for men and women is the same (1=yes; 0=no),There are periods of absence due to child care accounted for in pension benefits (1=yes; 0=no),There is legislation on sexual harassment in employment (1=yes; 0=no),There is legislation specifically addressing domestic violence (1=yes; 0=no),There is no legal provision that requires a married woman to obey her husband (1=yes; 0=no),There is paid parental leave (1=yes; 0=no)
A woman can apply for a passport in the same way as a man (1=yes; 0=no),1.000000,0.183954,0.247971,0.141631,0.199754,0.040175,-0.006841,-0.022190,0.318442,0.271888,...,-0.097865,0.146193,0.065538,0.169848,-0.068683,-0.019720,0.176352,0.119991,0.260615,0.201396
A woman can be head of household in the same way as a man (1=yes; 0=no),0.183954,1.000000,0.578965,0.552254,0.236002,0.397984,0.204908,0.233937,0.276563,0.278098,...,-0.042928,0.194204,0.364580,0.194502,-0.036641,0.049188,0.184610,0.267502,0.358622,0.200078
A woman can choose where to live in the same way as a man (1=yes; 0=no),0.247971,0.578965,1.000000,0.562946,0.364069,0.401504,0.257216,0.270823,0.408299,0.326784,...,-0.054729,0.255430,0.309436,0.242627,-0.112142,0.170227,0.235841,0.284756,0.448401,0.238241
A woman can get a job in the same way as a man (1=yes; 0=no),0.141631,0.552254,0.562946,1.000000,0.305459,0.635692,0.461624,0.482874,0.341238,0.311447,...,-0.008919,0.109934,0.422783,0.200598,-0.062276,0.115534,0.233945,0.270307,0.397944,0.176044
A woman can obtain a judgment of divorce in the same way as a man (1=yes; 0=no),0.199754,0.236002,0.364069,0.305459,1.000000,-0.005961,0.043951,0.050561,0.477923,0.361575,...,-0.031620,0.299209,0.002965,0.141156,-0.053995,0.155838,0.169823,0.153712,0.453183,0.231381
A woman can open a bank account in the same way as a man (1=yes; 0=no),0.040175,0.397984,0.401504,0.635692,-0.005961,1.000000,0.668932,0.762223,-0.103769,-0.078033,...,-0.000460,0.071731,0.582690,0.135496,-0.084620,0.052894,0.139768,0.184922,0.058750,0.112498
A woman can register a business in the same way as a man (1=yes; 0=no),-0.006841,0.204908,0.257216,0.461624,0.043951,0.668932,1.000000,0.807856,-0.096436,-0.072519,...,0.067903,0.116595,0.383671,0.114872,-0.037669,0.167556,0.135091,0.151284,0.028046,0.104548
A woman can sign a contract in the same way as a man (1=yes; 0=no),-0.022190,0.233937,0.270823,0.482874,0.050561,0.762223,0.807856,1.000000,-0.079095,-0.059479,...,0.084267,0.101914,0.397140,0.102812,-0.052849,0.175692,0.114105,0.148006,0.078149,0.085749
A woman can travel outside her home in the same way as a man (1=yes; 0=no),0.318442,0.276563,0.408299,0.341238,0.477923,-0.103769,-0.096436,-0.079095,1.000000,0.741722,...,0.028792,0.191982,-0.153265,0.123426,0.042226,0.060035,0.145185,0.135918,0.699969,0.112563
A woman can travel outside the country in the same way as a man (1=yes; 0=no),0.271888,0.278098,0.326784,0.311447,0.361575,-0.078033,-0.072519,-0.0

In [19]:
corr = GS_T.iloc[:,2:].corr(method='pearson')
np.fill_diagonal(corr.values, np.nan)
corr = corr.abs()
corr.style.background_gradient(cmap='coolwarm', axis=None)

,A woman can apply for a passport in the same way as a man (1=yes; 0=no),A woman can be head of household in the same way as a man (1=yes; 0=no),A woman can choose where to live in the same way as a man (1=yes; 0=no),A woman can get a job in the same way as a man (1=yes; 0=no),A woman can obtain a judgment of divorce in the same way as a man (1=yes; 0=no),A woman can open a bank account in the same way as a man (1=yes; 0=no),A woman can register a business in the same way as a man (1=yes; 0=no),A woman can sign a contract in the same way as a man (1=yes; 0=no),A woman can travel outside her home in the same way as a man (1=yes; 0=no),A woman can travel outside the country in the same way as a man (1=yes; 0=no),A woman can work at night in the same way as a man (1=yes; 0=no),A woman can work in a job deemed dangerous in the same way as a man (1=yes; 0=no),A woman can work in an industrial job in the same way as a man (1=yes; 0=no),A woman has the same rights to remarry as a man (1=yes; 0=no),Criminal penalties or civil remedies exist for sexual harassment in employment (1=yes; 0=no),Dismissal of pregnant workers is prohibited (1=yes; 0=no),Female and male surviving spouses have equal rights to inherit assets (1=yes; 0=no),Law mandates equal remuneration for females and males for work of equal value (1=yes; 0=no),Law prohibits discrimination in employment based on gender (1=yes; 0=no),Men and married women have equal ownership rights to immovable property (1=yes; 0=no),Paid leave is available to fathers (1=yes; 0=no),Paid leave of at least 14 weeks available to mothers (1=yes; 0=no),Sons and daughters have equal rights to inherit assets from their parents (1=yes; 0=no),The age at which men and women can retire with full pension benefits is the same (1=yes; 0=no),The age at which men and women can retire with partial pension benefits is the same (1=yes; 0=no),The government administers 100% of maternity leave benefits (1=yes; 0=no),The law grants spouses equal administrative authority over assets during marriage (1=yes; 0=no),The law prohibits discrimination in access to credit based on gender (1=yes; 0=no),The mandatory retirement age for men and women is the same (1=yes; 0=no),There are periods of absence due to child care accounted for in pension benefits (1=yes; 0=no),There is legislation on sexual harassment in employment (1=yes; 0=no),There is legislation specifically addressing domestic violence (1=yes; 0=no),There is no legal provision that requires a married woman to obey her husband (1=yes; 0=no),There is paid parental leave (1=yes; 0=no)
A woman can apply for a passport in the same way as a man (1=yes; 0=no),nan,0.183954,0.247971,0.141631,0.199754,0.040175,0.006841,0.022190,0.318442,0.271888,0.145822,0.065638,0.088844,0.053830,0.161929,0.268649,0.197729,0.111082,0.229015,0.039017,0.121214,0.247050,0.209813,0.110061,0.097865,0.146193,0.065538,0.169848,0.068683,0.019720,0.176352,0.119991,0.260615,0.201396
A woman can be head of household in the same way as a man (1=yes; 0=no),0.183954,nan,0.578965,0.552254,0.236002,0.397984,0.204908,0.233937,0.276563,0.278098,0.147845,0.321324,0.101797,0.510403,0.183636,0.255412,0.410907,0.160142,0.208964,0.381938,0.099031,0.060870,0.416312,0.043535,0.042928,0.194204,0.364580,0.194502,0.036641,0.049188,0.184610,0.267502,0.358622,0.200078
A woman can choose where to live in the same way as a man (1=yes; 0=no),0.247971,0.578965,nan,0.562946,0.364069,0.401504,0.257216,0.270823,0.408299,0.326784,0.094359,0.190510,0.084000,0.469564,0.223099,0.318096,0.279036,0.158597,0.257579,0.327092,0.030309,0.199721,0.267501,0.001751,0.054729,0.255430,0.309436,0.242627,0.112142,0.170227,0.235841,0.284756,0.448401,0.238241
A woman can get a job in the same way as a man (1=yes; 0=no),0.141631,0.552254,0.562946,nan,0.305459,0.635692,0.461624,0.482874,0.341238,0.311447,0.149824,0.293025,0.128831,0.451639,0.218866,0.209148,0.268580,0.152017,0.193012,0.441681,0.043254,0.111502,0.273258,0.046802,0.008919,0.109934